In [93]:
!pip install --upgrade pip
!pip install google-genai langchain_google_community langchain_huggingface sentence-transformers langchain chromadb pypdf faiss-cpu  langchain_community scikit-learn matplotlib seaborn numpy mistralai langchain-mistralai langchain_classic

In [2]:
import os
from getpass import getpass
from google.colab import drive
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS,Chroma
from langchain_classic.schema import Document
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader, GithubFileLoader, WeatherDataLoader
from langchain_google_community import GoogleDriveLoader,GoogleSearchAPIWrapper


In [4]:
import json
json_keys={}
with open("./keys.json","r") as f:
  json_keys=json.load(f)

In [8]:

os.environ["GOOGLE_API_KEY"] = json_keys["googleapi"]
os.environ["GOOGLE_CSE_ID"] = json_keys["cseapi"]
os.environ["MISTRAL_API_KEY"]=json_keys["mistralapi"]
os.environ["OPENWEATHERMAP_API_KEY"] = json_keys["openweatherapi"]
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./credentials.json"

In [13]:
# https://drive.google.com/drive/folders/15CvGCPEt59R6pQ5PLd6i6iidjVwCHWjZ?usp=drive_link
loader2 = GoogleDriveLoader(
    folder_id="15CvGCPEt59R6pQ5PLd6i6iidjVwCHWjZ",
    service_account_key="./service_account.json"
    )

In [14]:
docs = loader2.load()

In [15]:
print(docs)

[]


In [12]:
loader = GoogleSearchAPIWrapper()
search_list=["list popular cities in andhra pradesh, just list out?","what is debloater ?", "what is machine learning ?", "what is the top job notifications on IT roles from government sector ?", "how to remove google apps from android os using adb tools ?", "what is shizuku ?", "how to optimize the wondows 11 to speed up windows os ?"]
search_result = {}

for sq in search_list:
  search_result[sq]=loader.run(sq)

In [13]:
search_result["list popular cities in andhra pradesh ?"]

'Statistics ; 4, Nellore · Nellore, 794,783, 404,775 ; 5, Rajamahendravaram · East Godavari, 757,633, 269,122\xa0... List of Cities in Andhra Pradesh ; Vizianagaram, Vizianagaram, 29.27 km² ; Ongole, Prakasam, 132.4 km² ; Eluru, Eluru, 11.52 km² ; Nandyal, Nandyal, 33.67 km²\xa0... Geography. edit. Main articles: Geography of Andhra Pradesh and List of rivers in Andhra Pradesh ... City, Andhra Pradesh Special Economic Zone and several others\xa0... Major cities in Andhra Pradesh: · Hyderabad, India · Visakhapatnam, India · Vijayawada, India · Nellore, India · Warangal, India · Guntur, India · Ramagundam, India\xa0... Nov 14, 2025 ... Find information on the top 10 cities in Andhra Pradesh including their property price trends, connectivity, prime localities, and key facts\xa0... Jan 3, 2023 ... Rock Garden: Also called Silathoranam, Rock Garden is one of the major tourist attractions in Tirupati. · Sri Kalyana Venkateswara Swami Temple:\xa0... Apr 14, 2025 ... Andhra Nexus (@AndhraNexu

In [92]:
pip install pyowm

In [6]:
pip install -U langchain-google-genai

In [50]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview")

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

res = llm.invoke("list popular cities in andhra pradesh and make sure that just list out with seperated commas?").text
print(res)

Visakhapatnam, Vijayawada, Guntur, Nellore, Kurnool, Rajahmundry, Tirupati, Kakinada, Kadapa, Anantapur, Eluru, Vizianagaram, Chittoor, Machilipatnam, Ongole, Nandyal, Tenali


In [51]:
cities_names = [c.strip(" ") for c in res.split(",")]
print(cities_names)

['Visakhapatnam', 'Vijayawada', 'Guntur', 'Nellore', 'Kurnool', 'Rajahmundry', 'Tirupati', 'Kakinada', 'Kadapa', 'Anantapur', 'Eluru', 'Vizianagaram', 'Chittoor', 'Machilipatnam', 'Ongole', 'Nandyal', 'Tenali']


In [52]:
from langchain_community.utilities.openweathermap import OpenWeatherMapAPIWrapper
loader = WeatherDataLoader(client=OpenWeatherMapAPIWrapper(),places=cities_names)
data = loader.load()

In [53]:
for d in data:
  print(d)
  print("\n")

page_content='In Visakhapatnam, the current weather is as follows:
Detailed status: mist
Wind speed: 1.03 m/s, direction: 0°
Humidity: 88%
Temperature: 
  - Current: 18.94°C
  - High: 18.94°C
  - Low: 18.94°C
  - Feels like: 19.19°C
Rain: {}
Heat index: None
Cloud cover: 0%' metadata={'queried_at': datetime.datetime(2025, 12, 19, 17, 25, 27, 333595)}


page_content='In Vijayawada, the current weather is as follows:
Detailed status: haze
Wind speed: 0 m/s, direction: 0°
Humidity: 88%
Temperature: 
  - Current: 19.97°C
  - High: 19.97°C
  - Low: 19.97°C
  - Feels like: 20.32°C
Rain: {}
Heat index: None
Cloud cover: 40%' metadata={'queried_at': datetime.datetime(2025, 12, 19, 17, 25, 27, 525566)}


page_content='In Guntur, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 0.27 m/s, direction: 312°
Humidity: 54%
Temperature: 
  - Current: 18.6°C
  - High: 18.6°C
  - Low: 18.6°C
  - Feels like: 17.93°C
Rain: {}
Heat index: None
Cloud cover: 100%' metadata={'que

In [54]:
# print(search_result)

# for i in range(0,len(search_result),300):
#   data.append(Document(page_content=search_result[i:i+300]))

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=75, separators=["..","...",".","\n\n"])
chunks=splitter.split_documents(data)

print(len(chunks[0].page_content))

for idx, c in enumerate(chunks):
  print("Chunks {} ==>".format(idx+1),c.page_content)
  print("\n")

260
Chunks 1 ==> In Visakhapatnam, the current weather is as follows:
Detailed status: mist
Wind speed: 1.03 m/s, direction: 0°
Humidity: 88%
Temperature: 
  - Current: 18.94°C
  - High: 18.94°C
  - Low: 18.94°C
  - Feels like: 19.19°C
Rain: {}
Heat index: None
Cloud cover: 0%


Chunks 2 ==> In Vijayawada, the current weather is as follows:
Detailed status: haze
Wind speed: 0 m/s, direction: 0°
Humidity: 88%
Temperature: 
  - Current: 19.97°C
  - High: 19.97°C
  - Low: 19.97°C
  - Feels like: 20.32°C
Rain: {}
Heat index: None
Cloud cover: 40%


Chunks 3 ==> In Guntur, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 0.27 m/s, direction: 312°
Humidity: 54%
Temperature: 
  - Current: 18.6°C
  - High: 18.6°C
  - Low: 18.6°C
  - Feels like: 17.93°C
Rain: {}
Heat index: None
Cloud cover: 100%


Chunks 4 ==> In Nellore, the current weather is as follows:
Detailed status: broken clouds
Wind speed: 0.92 m/s, direction: 98°
Humidity: 79%
Temperature: 
  - Current:

In [55]:
vectordb = FAISS.from_documents(chunks, embedding)
retriver = vectordb.as_retriever(search_kwargs={"k":2})

In [56]:
query="what is just temperate at guntur ?"
res = retriver.invoke(query)
print(res[0].page_content)

In Guntur, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 0.27 m/s, direction: 312°
Humidity: 54%
Temperature: 
  - Current: 18.6°C
  - High: 18.6°C
  - Low: 18.6°C
  - Feels like: 17.93°C
Rain: {}
Heat index: None
Cloud cover: 100%


# Q2Q Query Optimation

In [65]:
def q2qOptimize(query):
  prompt_query_optimize = ChatPromptTemplate.from_template(
      '''
  Your a Query Optimizer tool,

  Rewrite the user query so that it retrieves the most relevant document chunks.
  Make it specific and explicit.

  User Query:
  {query}

  Optimized Query:

      '''
  )

  res = llm.invoke(prompt_query_optimize.format(query=query)).text
  return res

# Basic Rag Model

In [68]:
prompt = ChatPromptTemplate.from_template(
    '''
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}

    '''
)
doc_chain = create_stuff_documents_chain(llm,prompt)
rag_chain1 = create_retrieval_chain(retriver,doc_chain)


In [83]:
result = rag_chain1.invoke({"input":q2qOptimize(query)})
print(result)


{'input': 'Current weather conditions, temperature, humidity, precipitation, and 10-day forecast for Guntur, Andhra Pradesh, India.', 'context': [Document(id='61ac76c9-2534-460b-8740-040e3ef84760', metadata={'queried_at': datetime.datetime(2025, 12, 19, 17, 25, 27, 709326)}, page_content='In Guntur, the current weather is as follows:\nDetailed status: overcast clouds\nWind speed: 0.27 m/s, direction: 312°\nHumidity: 54%\nTemperature: \n  - Current: 18.6°C\n  - High: 18.6°C\n  - Low: 18.6°C\n  - Feels like: 17.93°C\nRain: {}\nHeat index: None\nCloud cover: 100%'), Document(id='2d332505-8ed6-434e-8d7c-aff9a1222669', metadata={'queried_at': datetime.datetime(2025, 12, 19, 17, 25, 28, 936866)}, page_content='In Anantapur, the current weather is as follows:\nDetailed status: scattered clouds\nWind speed: 2.88 m/s, direction: 101°\nHumidity: 42%\nTemperature: \n  - Current: 20.51°C\n  - High: 20.51°C\n  - Low: 20.51°C\n  - Feels like: 19.71°C\nRain: {}\nHeat index: None\nCloud cover: 44%')],

In [84]:
len(result)

3

In [85]:
print(result["answer"])

Based on the provided context, the current weather for Guntur is as follows:

*   **Current Weather Conditions:** Overcast clouds with 100% cloud cover and a wind speed of 0.27 m/s from 312°.
*   **Temperature:** 
    *   Current: 18.6°C
    *   High/Low: 18.6°C
    *   Feels like: 17.93°C
*   **Humidity:** 54%
*   **Precipitation:** No rain reported (Rain: {})
*   **10-day Forecast:** The provided context does not include information regarding a 10-day forecast.


# Memeory-Multi-Turn Rag Model

In [66]:
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain

In [88]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

prompt = ChatPromptTemplate.from_template(
    '''
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question:
{question}

    '''
)

rag_chain2 = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriver,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt}
)

In [89]:
optimized_query = q2qOptimize("What is the weather in guntur ?")
print(optimized_query)
result = rag_chain2.invoke({"question":optimized_query})
print(result["answer"])

**Optimized Query:**

Retrieve current meteorological data for Guntur, Andhra Pradesh, including current temperature, humidity levels, wind speed, precipitation probability, and the extended 7-day weather forecast.
Based on the provided context, the meteorological data for Guntur is as follows:

*   **Current Temperature:** 18.6°C
*   **Humidity:** 54%
*   **Wind Speed:** 0.27 m/s
*   **Precipitation Probability:** Not explicitly provided (Rain is listed as `{}`).
*   **7-Day Weather Forecast:** Not provided in the context.


In [91]:
while(True):
  query=input("Enter the Query about weather Forecasting: ")
  if query.lower()=="exit" or query.lower()=="quit":
    break
  optimized_query = q2qOptimize(query)
  print(optimized_query)
  result = rag_chain2.invoke({"question":optimized_query})
  print(result["answer"])